In [57]:
from pathlib import Path
from src.data import make_dataset
from dotenv import find_dotenv, load_dotenv
import os
from scipy import io
import numpy as np

In [18]:
load_dotenv(find_dotenv())

True

In [19]:
project_dir = Path(os.getenv("project_dir")).resolve()
input_filepath = project_dir / "data" / "raw"
output_filepath = project_dir / "data" / "processed"
interim_filepath = project_dir / "data" / "interim"

In [51]:
classes = []
x_train_links = []
y_train = []
with open(input_filepath / "fru92_lists" / "fru_train.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        if int(class_no) == len(classes):
            classes.append(name)
        
        x_train_links.append("fru92_images/" + f_path)
        y_train.append(int(class_no))

class_base = len(classes)
with open(input_filepath / "veg200_lists" / "veg_train.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        if int(class_no)+class_base == len(classes):
            classes.append(name)
        
        x_train_links.append("veg200_images/" + f_path)
        y_train.append(int(class_no)+class_base)

In [52]:
x_test_links = []
y_test = []

x_val_links = []
y_val = []
with open(input_filepath / "fru92_lists" / "fru_test.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        x_test_links.append("fru92_images/" + f_path)
        y_test.append(int(class_no))
        
with open(input_filepath / "veg200_lists" / "veg_test.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        x_test_links.append("veg200_images/" + f_path)
        y_test.append(int(class_no)+class_base)
        
with open(input_filepath / "fru92_lists" / "fru_val.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        x_val_links.append("fru92_images/" + f_path)
        y_val.append(int(class_no))
        
with open(input_filepath / "veg200_lists" / "veg_val.txt") as f:
    for line in f.readlines():
        f_path, class_no = line.split(" ")
        name = f_path.split("/")[0]
        
        x_val_links.append("veg200_images/" + f_path)
        y_val.append(int(class_no)+class_base)
        

In [ ]:
def links_to_images(links):
    images = []
    for link in links:
        images.append(link_to_image(link))
    return np.array(images)

def link_to_image(link):
    pass